In [1]:
## ---- Partition Cell Set Network Heatmap Module Analysis ---- 0681.01.00
## Load Analysis Parameters (Parm1)
## Load Network Utilities
## Loop Through Each Partition and Perform Analysis as Follows:
## Load Partition Cell Set
## Plot Module Gene Heatmaps, Heavily Modified version of 681.0x.00

In [2]:
## Create a Working Input and Output Data Directory, If Id Does Not Exist
parentdir <- '/gpfs/group/torkamani/devans/'
datdir <- paste(parentdir, 'CDC2', sep = '')
if (!file.exists(datdir)) {
    dir.create(datdir)
}
setwd(datdir)

In [3]:
## Read the parameters file
ps <- read.table(file = 'parms.txt', stringsAsFactors = FALSE, header = TRUE)

In [5]:
## Load Monocle3 and Seurat Libraries
library(monocle3)
library(Seurat)
library(dplyr)
library(magrittr)
library(ggplot2)
library(gridExtra)
library(Matrix)
library(rhdf5)
library(grid)
library(igraph)
library(corpcor)
library(fdrtool)
library(readxl)
library(mixOmics)

In [6]:
## Load modified version of Monocle 3 dot plot (adds as.ordered type)
source('~/code/CSC.Run2/0700.00.00.Carotid.SC.Utilities.01.r')

In [7]:
## Read the previously preprocessed downsampled cell set data object
down_stdycds <- readRDS(file = paste(ps$outdir,
            'Aggregated.downsampled.QC.NoDoublets.Repartitioned.rds', sep = ''))

In [8]:
## Build a gene short name to gene id (Ensembl) lookup
short2geneid <- fData(down_stdycds)@rownames
names(short2geneid) <- fData(down_stdycds)@listData$gene_short_name

In [9]:
## Build a gene id (Ensembl) to gene short name lookup
geneid2short <- fData(down_stdycds)@listData$gene_short_name
names(geneid2short) <- fData(down_stdycds)@rownames

In [10]:
## Create variables for how cells sets are organized
cellgrps <- c('healthy', 'diseased', 'healthy', 'diseased', 'healthy', 'diseased')
cellpats <- c('ID Number 1', 'ID Number 1', 'ID Number 2', 'ID Number 2', 'ID Number 3', 'ID Number 3')

In [11]:
## Define and Assign Cell Types
celltypes6 <- c('1-Macrophages',
                '2-Endothelial Cells',
                '3-VSMCs',
                '4-Natural Killer Cells',
                '5-Cytotoxic T Lymphocytes',
                '6-B Lymphocytes')

In [209]:
## Get the pairs of module genes from EC (1 and 36) and VSCM (13 and 31) networks
## Set the q-value Cutoff
diff_exp_qval_cut <- 0.05
## EC Genes First
ec_netmod_file <- paste(datdir, '/', ps$outdir, celltypes6[2],
                          '/Dense.20k.Network.Module.Genes.Rev1.txt', sep = '')
ec_netmod <- read.table(ec_netmod_file, header = TRUE, sep = '\t',
                          stringsAsFactors = FALSE, quote = '')[, c(1, 3:9)]
ec_modgenes1 <- ec_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 1) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
ec_modgenes2 <- ec_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 2) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
ec_modgenes6 <- ec_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 6) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
ec_modgenes35 <- ec_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 35) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
ec_modgenes36 <- ec_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 36) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)

ec_modgenes1_ids <- short2geneid[ec_modgenes1$gene_short_name]
ec_modgenes36_ids <- short2geneid[ec_modgenes36$gene_short_name]

ec_modgenes_1_36 <- c(ec_modgenes1$gene_short_name,
                      ec_modgenes36$gene_short_name)
ec_modgenes_1_36_ids <- short2geneid[ec_modgenes_1_36]

ec_modgenes_1_35_36 <- c(ec_modgenes1$gene_short_name,
                         ec_modgenes35$gene_short_name,
                         ec_modgenes36$gene_short_name)

ec_modgenes_1_2_6_35_36 <- c(ec_modgenes1$gene_short_name,
                         ec_modgenes2$gene_short_name, 
                         ec_modgenes6$gene_short_name, 
                         ec_modgenes35$gene_short_name,
                         ec_modgenes36$gene_short_name)

ec_modgenes_1_35_36_ids <-  short2geneid[ec_modgenes_1_35_36]
ec_modgenes_1_2_6_35_36_ids <-  short2geneid[ec_modgenes_1_2_6_35_36]


## VSMC Genes Seconds
vsmc_netmod_file <- paste(datdir, '/', ps$outdir, celltypes6[3],
                          '/Dense.20k.Network.Module.Genes.Rev1.txt', sep = '')
vsmc_netmod <- read.table(vsmc_netmod_file, header = TRUE, sep = '\t',
                          stringsAsFactors = FALSE, quote = '')[, c(1, 3:9)]
vsmc_modgenes9 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 9) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes11 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 11) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes13 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 13) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes24 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 24) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes28 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 28) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes30 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 30) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes31 <- vsmc_netmod %>% filter(de_qval < diff_exp_qval_cut) %>% 
                              filter(module_id == 31) %>% 
                              dplyr::select(gene_short_name, de_norm_eff, de_qval)
vsmc_modgenes_13_31 <- c(vsmc_modgenes13$gene_short_name, vsmc_modgenes31$gene_short_name)
vsmc_modgenes13_ids <- short2geneid[vsmc_modgenes13$gene_short_name]
vsmc_modgenes31_ids <- short2geneid[vsmc_modgenes31$gene_short_name]
vsmc_modgenes_13_31_ids <- short2geneid[vsmc_modgenes_13_31]
vsmc_modgenes_11_30 <- c(vsmc_modgenes11$gene_short_name,
                         vsmc_modgenes30$gene_short_name)
vsmc_modgenes_11_35_ids <- short2geneid[vsmc_modgenes_11_30]

vsmc_modgenes_11_13_30_31 <- c(vsmc_modgenes11$gene_short_name,
                              vsmc_modgenes13$gene_short_name,
                              vsmc_modgenes30$gene_short_name,
                              vsmc_modgenes31$gene_short_name)

vsmc_modgenes_9_11_13_30_31 <- c(vsmc_modgenes9$gene_short_name,
                              vsmc_modgenes11$gene_short_name,
                              vsmc_modgenes13$gene_short_name,
                              vsmc_modgenes30$gene_short_name,
                              vsmc_modgenes31$gene_short_name)

vsmc_modgenes_9_11_13_24_28_30_31 <- c(vsmc_modgenes9$gene_short_name,
                              vsmc_modgenes11$gene_short_name,
                              vsmc_modgenes13$gene_short_name,
                              vsmc_modgenes24$gene_short_name,
                              vsmc_modgenes28$gene_short_name,
                              vsmc_modgenes30$gene_short_name,
                              vsmc_modgenes31$gene_short_name)

vsmc_modgenes_9_11_13_24_28_30_31_plus2 <- c(vsmc_modgenes_9_11_13_24_28_30_31, 'CCL4', 'IGKC')


vsmc_modgenes_11_13_30_31_ids <- short2geneid[vsmc_modgenes_11_13_30_31]
vsmc_modgenes_9_11_13_30_31_ids <- short2geneid[vsmc_modgenes_9_11_13_30_31] 
vsmc_modgenes_9_11_13_24_28_30_31_ids <- short2geneid[vsmc_modgenes_9_11_13_24_28_30_31] 
vsmc_modgenes_9_11_13_24_28_30_31_plus2_ids <- short2geneid[vsmc_modgenes_9_11_13_24_28_30_31_plus2] 

## These variables are the same as above
vsmc_modgenes_11_30a <- c('TMSB4X', 'FXYD5', 'MYH10', 'RAMP1', 'ANXA2',
                  'TSC22D3', 'F2R', 'SFRP4', 'ECM1', 'ABHD14A',
                  'JDP2', 'DDIT4', 'VAT1', 'ABLIM1', 'S100A10',
                  'ZNF385D', 'FRZB', 'MAP1B', 'ZNF385B', 'GAP43',
                  'PDE5A', 'PRSS35', 'CCDC68', 'SH3BGRL3', 'UXT',
                  'CDKN2C', 'LMO7', 'ENPP2', 'NDNF', 'MCUB', 'HTR2B',
                  'MBNL1-AS1', 'TBC1D12', 'PTN', 'IL17D', 'SFT2D1',
                  'IGSF10', 'BHLHE41', 'NUCKS1', 'HACD3', 'IL18',
                  'TPPP3', 'EMP3', 'CLIC2', 'ATF5', 'MSC-AS1', 'FBLN2',
                  'LTBP4', 'F10', 'FSTL1', 'FBN1', 'TNXB', 'PCOLCE2',
                  'PLAT', 'PLPP3', 'SLIT3', 'CD248', 'AKAP12', 'PLBD1',
                  'SEMA3C', 'MFAP5', 'C17orf58', 'METRNL', 'UGDH',
                  'HSD11B1', 'TGFBR3', 'TSKU', 'FLRT2', 'SEMA3B',
                  'PI16', 'RAB32', 'DCLK1', 'CD55', 'LINC01133',
                  'FGFR1', 'CSF1', 'SLC16A7', 'NT5E', 'ADAMTSL4',
                  'HTRA3', 'FAM177A1', 'C1QTNF3', 'PPP2R2A', 'GPC1',
                  'EBF2', 'AMOTL2', 'NTM', 'CMTM3', 'CADM3')
vsmc_modgenes_11_30a_ids <- short2geneid[vsmc_modgenes_11_30]

ec_modgenes_35a <- c('FEZ1', 'OLFM1', 'ARHGAP18', 'COL4A1', 'MEDAG', 'COL4A2',
             'PDGFD', 'IGFBP7', 'RGS3', 'PGF', 'ITGA6', 'MGLL', 'ACKR3',
             'ANGPT2', 'PLXNA4', 'EMCN', 'TIMP3', 'GSN', 'FILIP1', 'LAMB1',
             'ELK3', 'ARPC2', 'NES', 'MXD4', 'PLVAP', 'ESM1', 'CITED4',
             'TMED3', 'EIF4EBP1', 'RFLNB')
ec_modgenes_35a_ids <- short2geneid[ec_modgenes_35a]

[1] 232

## EC EMT + TNF  (Modules 1 and 36)

In [118]:
## Get the Expression Set for EC Module 1 and 36 DE Genes
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes_1_36_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

table(colData(ecmod_cds)$sidecolor)
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.EMT_TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network EMT and TNF \n(Module 1 and 36) Genes Cells Pre-Clustered'

f2 <- '/Heatmap.EC.EMT_TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network EMT and TNF\n (Module 1 and 36) Genes Cells Fully Clustered'

f3 <- '/DotPlot.EC.EMT_TNF.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')
ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 18)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 18)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## Dot Plot Based on Genes Order
specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 18)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

## VSMC EMT + TNF  (Modules 13 and 31)

In [119]:
## Get the Expression Set for VSMC Module 13 and 31 DE Genes
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes_13_31_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

table(colData(vsmcmod_cds)$sidecolor)
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## This Code Plots the EC Cell Type with EMT and TNF Network Module Genes
f1 <- '/Heatmap.VSMC.EMT_TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network EMT and TNF (Module 13 and 31)\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.EMT_TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network EMT and TNF (Module 13 and 31)\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.EMT_TNF.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust],
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 18)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 18)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 18)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

## EC EMT (Module 1) 

In [121]:
## Get the Expression Set for EC Module 1 (EMT)
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes1_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

table(colData(ecmod_cds)$sidecolor)
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.EMT.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network EMT\n(Module 1) Genes, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.EMT.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network EMT\n (Module 1) Genes, Cells Fully Clustered'

f3 <- '/DotPlot.EC.EMT.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 18)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_row <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 18)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2, 
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 18)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

## EC TNF (Module 36) 

In [122]:
## Get the Expression Set for EC Module 36 (TNF)
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes36_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

table(colData(ecmod_cds)$sidecolor)
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network TNF\n(Module 36) Genes, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network TNF\n(Module 36) Genes, Cells Fully Clustered'

f3 <- '/DotPlot.EC.TNF.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 10, height = 6)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_row <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f2, width = 10, height = 6)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()


specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 4)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

## EC EMT (Module 1) New Filter (No Filter) Applied

In [123]:
## Get the Expression Set for EC Module 1 (EMT), New Filter
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes1_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

table(colData(ecmod_cds)$sidecolor)
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.EMT.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network EMT\n(Module 1) Genes, No Filter, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.EMT.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network EMT\n (Module 1) Genes, No Filter, Cells Fully Clustered'

f3 <- '/DotPlot.EC.EMT.NoFilt.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 0) * 1 ## (expdat_ecmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 18)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_row <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 18)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2, 
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 18)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

## EC EMT (Module 36) New Filter (No Filter) Applied

In [124]:
## Get the Expression Set for EC Module 36 (TNF), New Filter
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes36_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

table(colData(ecmod_cds)$sidecolor)
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.TNF.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network TNF\n(Module 36) Genes, No Filter, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.TNF.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network TNF\n(Module 36) Genes, No Filter, Cells Fully Clustered'

f3 <- '/DotPlot.EC.TNF.NoFilt.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 0) * 1 ## (expdat_ecmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 10, height = 6)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_row <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f2, width = 10, height = 6)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()


specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 4)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

## VSMC EMT (Module 13) 

In [125]:
## Get the Expression Set for VSMC Module 13 DE Genes
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes13_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

table(colData(vsmcmod_cds)$sidecolor)
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## This Code Plots the EC Cell Type with EMT and TNF Network Module Genes
f1 <- '/Heatmap.VSMC.EMT.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network EMT (Module 13)\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.EMT.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network EMT (Module 13)\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.EMT.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust],
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 16)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 16)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 16)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

## VSMC TNF (Module 31) 

In [126]:
## Get the Expression Set for VSMC Module 31 DE Genes
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes31_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

table(colData(vsmcmod_cds)$sidecolor)
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## This Code Plots the EC Cell Type with EMT and TNF Network Module Genes
f1 <- '/Heatmap.VSMC.TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network TNF (Module 31)\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.TNF.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network TNF (Module 31)\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.TNF.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust],
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 16)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 16)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 16)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

## VSMC EMT (Module 13) New Filter (No Filter) Applied

In [127]:
## Get the Expression Set for VSMC Module 13 DE Genes, New Filter
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes13_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

table(colData(vsmcmod_cds)$sidecolor)
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## This Code Plots the EC Cell Type with EMT and TNF Network Module Genes
f1 <- '/Heatmap.VSMC.EMT.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network EMT (Module 13)\nGenes, No Filter, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.EMT.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network EMT (Module 13)\nGenes, No Filter, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.EMT.NoFilt.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 0) * 1 ## (expdat_vsmcmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust],
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 16)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 16)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 16)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

## VSMC TNF (Module 31) New Filter (No Filter) Applied

In [128]:
## Get the Expression Set for VSMC Modules 31 DE Genes, New Filter
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes13_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

table(colData(vsmcmod_cds)$sidecolor)
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## This Code Plots the EC Cell Type with EMT and TNF Network Module Genes
f1 <- '/Heatmap.VSMC.TNF.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network TNF (Module 31)\nGenes, No Filter, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.TNF.NoFilt.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network TNF (Module 31)\nGenes, No Filter, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.TNF.NoFilt.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 0) * 1 ## (expdat_vsmcmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust],
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 16)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 16)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 16)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

## EC Unknown Process (Modules 1, 35, 36)
### Superceded by updated code below

In [171]:
## Get the Expression Set for EC 1, 35, 36 DE Genes
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes_1_35_36_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

table(colData(ecmod_cds)$sidecolor)
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.Mod1.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev1.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network \nModules 1, 35 and 36 Genes, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.Mod1.35.3.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev1.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network \nModule  1, 35 and 36 Genes, Cells Fully Clustered'

f3 <- '/DotPlot.EC.1.35.36.Genes.Rev1.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')
ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 24)
cim_row <-cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both',
    col.sideColors = colornames[parthclust],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 24)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## Dot Plot Based on Genes Order
specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 23)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

## EC Unknown Process (Modules 1, 35, 36)
### Updated code

In [152]:
## Get the Expression Set for EC 1, 35, 36 DE Genes
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes_1_35_36_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
# colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
# colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'

## Down Sample
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

## Print the down sampled data set sizes
table(colData(ecmod_cds)$sidecolor)

## Get the expresion matrix
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.Mod1.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev2.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network \nModules 1, 35 and 36 Genes, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.Mod1.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev2.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network \nModule  1, 35 and 36 Genes, Cells Fully Clustered'

f3 <- '/DotPlot.EC.Mod1.35.36.Genes.Rev2.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

f4 <- '/Heatmap.EC.Mod1.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.cut.Rev4.pdf'
f4 <- paste(datdir, '/', ps$outdir, celltypes6[p], f4, sep = '')
title4 <-  'VSMC Network Modules  1, 35 and 36\nGenes, Cells Fully Clustered with Cuts'

f5 <- '/DotPlot.EC.Mod1.35.36.Genes.cuts.Rev4.pdf'
f5 <- paste(datdir, '/', ps$outdir, celltypes6[p], f5, sep = '')

## Prepare for pre-clustering
ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')

## Transform the data
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 1) * 1

## Set up clustering parameters
## Distance method, cluster method, heatmap color scheme, legend
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

## Individually cluster the columns of the two cell sets
ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')
ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## This is the combined (concantenated) preclustered cell types
## which are plotted with genes fully clustered (check plot)
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)

## This is just a check plot to the screen
# cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row',
#     col.sideColors = colornames[parthclust],
#     symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 24)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_save <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## This is a check plot to the screen and it should match the above plot perfectly
# cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[colnames(cim_save$mat)], 
#     symkey = FALSE)

## Repeated from above, this time to a file (make sure to cluster again to plot cell dendrogram)
pdf(f2, width = 8, height = 24)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- short2geneid[cim_save$row.names]
pdf(f3, width = 2.75, height = 23)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

## Set up for plot with special cut regions
cim_o <- colnames(cim_save$mat)
cut0 <- 1
cut1 <- floor(length(cim_o) * (2.15) / 7.375)
cut2 <- floor(length(cim_o) * (3 + 3 / 8) / 7.375)
cut3 <- floor(length(cim_o) * (4 + 3 / 4) / 7.375)
cut4 <- floor(length(cim_o) * (5 + 1 / 4) / 7.375)
cut5 <- floor(length(cim_o) * (6 + 3 / 4) / 7.375)
cut6 <- length(cim_o)
wcut1 <- which(colnames(expdatbo) %in% cim_o[cut0:cut1])
wcut2 <- which(colnames(expdatbo) %in% cim_o[(cut1 + 1):cut2])
wcut3 <- which(colnames(expdatbo) %in% cim_o[(cut2 + 1):cut3])
wcut4 <- which(colnames(expdatbo) %in% cim_o[(cut3 + 1):cut4])
wcut5 <- which(colnames(expdatbo) %in% cim_o[(cut4 + 1):cut5])
wcut6 <- which(colnames(expdatbo) %in% cim_o[(cut5 + 1):cut6])
wcuts <- c(wcut1, wcut2, wcut3, wcut4, wcut5, wcut6)
cell_subg_cols <- vector(mode = 'character', length = length(cim_o))
cell_subg_names <- vector(mode = 'character', length = length(cim_o))
cell_subg_cols[wcut1] <- '#404040' # '1gray'
cell_subg_names[wcut1] <- '1gray'
cell_subg_cols[wcut2] <- '#606060' # '2gray'
cell_subg_names[wcut2] <- '2gray'
cell_subg_cols[wcut3] <- '#808080' # '3gray'
cell_subg_names[wcut3] <- '3gray'
cell_subg_cols[wcut4] <- '#A0A0A0' # '4gray'
cell_subg_names[wcut4] <- '4gray'
cell_subg_cols[wcut5] <- '#C0C0C0' # '5gray'
cell_subg_names[wcut5] <- '5gray'
cell_subg_cols[wcut6] <- '#E0E0E0' # '6gray'
cell_subg_names[wcut6] <- '6gray'
names(cell_subg_cols) <- cim_o

## Plot the same fully clustered plot, this time without reclustering the cells,
# but coloring the horizontal color bar according to the cuts
pdf(f4, width = 8, height = 24)
cim_cells <- cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth,
    clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title4,
    col.sideColors = cell_subg_cols[wcuts],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_save$row.names]
colData(ecmod_cds)$subgroups[wcuts] <- cell_subg_names[wcuts]
pdf(f5, width = 3.25, height = 23)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "subgroups",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

## VSMC Unknown Process (Modules 11, 13,  30, and 31)
### Superceded by code below (now includes module 9 ) 

In [69]:
## Get the Expression Set for VSMC Module 11, 13, 30, and 31 DE Genes
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes_11_13_30_31_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

table(colData(vsmcmod_cds)$sidecolor)
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## This Code Plots the EC Cell Type with EMT and TNF Network Module Genes
f1 <- '/Heatmap.VSMC.Mod11.13.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev2.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network Modules 11, 13, 30, 31\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.Mod11.13.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev2.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network Modules 11, 13, 30, 31\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.Mod11.13.30.31.Genes.Rev2.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined cell types, plotted with only genes fully clustered
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust],
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 26)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 26)
cim_row <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_row$row.names]
# str(specialglist)
pdf(f3, width = 2.75, height = 25)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

cim_o <- colnames(cim_row$mat)
cut0 <- 1
cut1 <- floor(length(cim_o) * (1 + 1/8) / 5.525)
cut2 <- floor(length(cim_o) * (1 + 11/16) / 5.525)
cut3 <- floor(length(cim_o) * (2 + 7/16) / 5.525)
cut4 <- floor(length(cim_o) * (3 + 3/16) / 5.525)
cut5 <- floor(length(cim_o) * (5 + 1/16) / 5.525)
cut6 <- length(cim_o)
wcut1 <- which(colnames(expdatbo) %in% cim_o[cut0:cut1])
wcut2 <- which(colnames(expdatbo) %in% cim_o[(cut1 + 1):cut2])
wcut3 <- which(colnames(expdatbo) %in% cim_o[(cut2 + 1):cut3])
wcut4 <- which(colnames(expdatbo) %in% cim_o[(cut3 + 1):cut4])
wcut5 <- which(colnames(expdatbo) %in% cim_o[(cut4 + 1):cut5])
wcut6 <- which(colnames(expdatbo) %in% cim_o[(cut5 + 1):cut6])
wcuts <- c(wcut1, wcut2, wcut3, wcut4, wcut5, wcut6)
cell_subg_cols <- vector(mode = 'character', length = length(cim_o))
cell_subg_names <- vector(mode = 'character', length = length(cim_o))
cell_subg_cols[wcut1] <- '#404040' # '1gray'
cell_subg_names[wcut1] <- '1gray'
cell_subg_cols[wcut2] <- '#606060' # '2gray'
cell_subg_names[wcut2] <- '2gray'
cell_subg_cols[wcut3] <- '#808080' # '3gray'
cell_subg_names[wcut3] <- '3gray'
cell_subg_cols[wcut4] <- '#A0A0A0' # '4gray'
cell_subg_names[wcut4] <- '4gray'
cell_subg_cols[wcut5] <- '#C0C0C0' # '5gray'
cell_subg_names[wcut5] <- '5gray'
cell_subg_cols[wcut6] <- '#E0E0E0' # '6gray'
cell_subg_names[wcut6] <- '6gray'
names(cell_subg_cols) <- cim_o

f2 <- '/Heatmap.VSMC.Mod11.13.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.cut.Rev4.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network Modules 11, 13, 30, 31\nGenes, Cells Fully Clustered with Cuts'


f3 <- '/DotPlot.VSMC.Mod11.13.30.31.Genes.cuts.Rev4.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

## Repeated from above, this time to a file
pdf(f2, width = 8, height = 26)
cim_cells <- cim(expdatbo[, wcuts], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title2,
    col.sideColors = cell_subg_cols[wcuts],
    symkey = FALSE)
dev.off()


specialglist <- specialglist <- short2geneid[cim_cells$row.names]
pdf(f3, width = 3.25, height = 25)
colData(vsmcmod_cds)$subgroups[wcuts] <- cell_subg_names[wcuts]
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "subgroups",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

## VSMC Unknown Process (Modules 9, 11, 13, 30, and 31)
### Updated code

In [146]:
## Get the Expression Set for VSMC Module 9, 11, 13, 30, and 31 DE Genes
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes_9_11_13_30_31_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
## colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
## colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'

## Down Sample
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

## Print the down sampled data set sizes
table(colData(vsmcmod_cds)$sidecolor)

## Get the expresion matrix
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## Set up the output file names and plot titles
f1 <- '/Heatmap.VSMC.Mod9.11.13.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev2.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network Modules 9, 11, 13, 30, 31\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.Mod9.11.13.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev2.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network Modules 9, 11, 13, 30, 31\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.Mod9.11.13.30.31.Genes.Rev2.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

f4 <- '/Heatmap.VSMC.Mod9.11.13.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.cut.Rev4.pdf'
f4 <- paste(datdir, '/', ps$outdir, celltypes6[p], f4, sep = '')
title4 <-  'VSMC Network Modules 9, 11, 13, 30, 31\nGenes, Cells Fully Clustered with Cuts'

f5 <- '/DotPlot.VSMC.Mod9.11.13.30.31.Genes.cuts.Rev4.pdf'
f5 <- paste(datdir, '/', ps$outdir, celltypes6[p], f5, sep = '')

## Prepare for pre-clustering
vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')

## Transform the data
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1

## Set up clustering parameters
## Distance method, cluster method, heatmap color scheme, legend
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

## Individually cluster the columns of the two cell sets
vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined (concantenated) preclustered cell types
## which will are plotted with genes fully clustered (check plot)
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)

## This is just a check plot to the screen
# cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[parthclust],
#     symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 26)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_save <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## This is a check plot to the screen and it should match the above plot perfectly
# cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[colnames(cim_save$mat)], 
#     symkey = FALSE)

## Repeated from above, this time to a file (make sure to cluster again to plot cell dendrogram)
pdf(f2, width = 8, height = 26)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- short2geneid[cim_save$row.names]
pdf(f3, width = 2.75, height = 30)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

## Set up for plot with special cut regions
cim_o <- colnames(cim_save$mat)
cut0 <- 1
cut1 <- floor(length(cim_o) * (15 / 16) / 7.1)
cut2 <- floor(length(cim_o) * (1 + 11 / 16) / 7.1)
cut3 <- floor(length(cim_o) * (2 + 1 / 4) / 7.1)
cut4 <- floor(length(cim_o) * (3 + 5 / 8) / 7.1)
cut5 <- floor(length(cim_o) * (4 + 15 / 16) / 7.1)
cut6 <- length(cim_o)
wcut1 <- which(colnames(expdatbo) %in% cim_o[cut0:cut1])
wcut2 <- which(colnames(expdatbo) %in% cim_o[(cut1 + 1):cut2])
wcut3 <- which(colnames(expdatbo) %in% cim_o[(cut2 + 1):cut3])
wcut4 <- which(colnames(expdatbo) %in% cim_o[(cut3 + 1):cut4])
wcut5 <- which(colnames(expdatbo) %in% cim_o[(cut4 + 1):cut5])
wcut6 <- which(colnames(expdatbo) %in% cim_o[(cut5 + 1):cut6])
wcuts <- c(wcut1, wcut2, wcut3, wcut4, wcut5, wcut6)
cell_subg_cols <- vector(mode = 'character', length = length(cim_o))
cell_subg_names <- vector(mode = 'character', length = length(cim_o))
cell_subg_cols[wcut1] <- '#404040' # '1gray'
cell_subg_names[wcut1] <- '1gray'
cell_subg_cols[wcut2] <- '#606060' # '2gray'
cell_subg_names[wcut2] <- '2gray'
cell_subg_cols[wcut3] <- '#808080' # '3gray'
cell_subg_names[wcut3] <- '3gray'
cell_subg_cols[wcut4] <- '#A0A0A0' # '4gray'
cell_subg_names[wcut4] <- '4gray'
cell_subg_cols[wcut5] <- '#C0C0C0' # '5gray'
cell_subg_names[wcut5] <- '5gray'
cell_subg_cols[wcut6] <- '#E0E0E0' # '6gray'
cell_subg_names[wcut6] <- '6gray'
names(cell_subg_cols) <- cim_o

## Plot the same fully clustered plot, this time without reclustering the cells,
# but coloring the horizontal color bar according to the cuts
pdf(f4, width = 8, height = 26)
cim_cells <- cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth,
    clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title4,
    col.sideColors = cell_subg_cols[wcuts],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_save$row.names]
colData(vsmcmod_cds)$subgroups[wcuts] <- cell_subg_names[wcuts]
pdf(f5, width = 3.25, height = 32)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "subgroups",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

## VSMC Unknown Process (Modules 9, 11, 13, 24, 28, 30, and 31)¶
### Updated code

In [226]:
## Get the Expression Set for VSMC Module 9, 11, 13, 24, 28, 30, and 31 DE Genes
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes_9_11_13_24_28_30_31_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
## colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
## colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'

## Down Sample
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

## Print the down sampled data set sizes
table(colData(vsmcmod_cds)$sidecolor)

## Get the expresion matrix
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## Set up the output file names and plot titles
f1 <- '/Heatmap.VSMC.Mod9.11.13.24.28.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev3.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network Modules 9, 11, 13, 24, 28, 30, 31\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.Mod9.11.13.24.28.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev3.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network Modules 9, 11, 24, 28, 13, 30, 31\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.Mod9.11.13.24.28.30.31.Genes.Rev3.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

f4 <- '/Heatmap.VSMC.Mod9.11.13.24.28.30.31.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.cut.Rev6.pdf'
f4 <- paste(datdir, '/', ps$outdir, celltypes6[p], f4, sep = '')
title4 <-  'VSMC Network Modules 9, 11, 24, 28, 13, 30, 31\nGenes, Cells Fully Clustered with Cuts'

f5 <- '/DotPlot.VSMC.Mod9.11.13.24.28.30.31.Genes.cuts.Rev6.pdf'
f5 <- paste(datdir, '/', ps$outdir, celltypes6[p], f5, sep = '')

## Prepare for pre-clustering
vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')

## Transform the data
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1

## Set up clustering parameters
## Distance method, cluster method, heatmap color scheme, legend
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

## Individually cluster the columns of the two cell sets
vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined (concantenated) preclustered cell types
## which will are plotted with genes fully clustered (check plot)
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)

## This is just a check plot to the screen
# cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[parthclust],
#     symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 30)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_save <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## This is a check plot to the screen and it should match the above plot perfectly
# cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[colnames(cim_save$mat)], 
#     symkey = FALSE)

## Repeated from above, this time to a file (make sure to cluster again to plot cell dendrogram)
pdf(f2, width = 8, height = 30)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- short2geneid[cim_save$row.names]
pdf(f3, width = 2.75, height = 34)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

## Set up for plot with special cut regions
cim_o <- colnames(cim_save$mat)
cut0 <- 1
cut1 <- floor(length(cim_o) * (1.100) / 5.626)
cut2 <- floor(length(cim_o) * (1.561) / 5.626)
cut3 <- floor(length(cim_o) * (2.169) / 5.626)
cut4 <- floor(length(cim_o) * (3.178) / 5.626)
cut5 <- floor(length(cim_o) * (3.634) / 5.626)
cut6 <- floor(length(cim_o) * (4.100) / 5.626)
cut7 <- length(cim_o)
wcut1 <- which(colnames(expdatbo) %in% cim_o[cut0:cut1])
wcut2 <- which(colnames(expdatbo) %in% cim_o[(cut1 + 1):cut2])
wcut3 <- which(colnames(expdatbo) %in% cim_o[(cut2 + 1):cut3])
wcut4 <- which(colnames(expdatbo) %in% cim_o[(cut3 + 1):cut4])
wcut5 <- which(colnames(expdatbo) %in% cim_o[(cut4 + 1):cut5])
wcut6 <- which(colnames(expdatbo) %in% cim_o[(cut5 + 1):cut6])
wcut7 <- which(colnames(expdatbo) %in% cim_o[(cut6 + 1):cut7])
wcuts <- c(wcut1, wcut2, wcut3, wcut4, wcut5, wcut6, wcut7)
cell_subg_cols <- vector(mode = 'character', length = length(cim_o))
cell_subg_names <- vector(mode = 'character', length = length(cim_o))
cell_subg_cols[wcut1] <- '#202020' # '1gray'
cell_subg_names[wcut1] <- '1gray'
cell_subg_cols[wcut2] <- '#404040' # '2gray'
cell_subg_names[wcut2] <- '2gray'
cell_subg_cols[wcut3] <- '#606060' # '3gray'
cell_subg_names[wcut3] <- '3gray'
cell_subg_cols[wcut4] <- '#808080' # '4gray'
cell_subg_names[wcut4] <- '4gray'
cell_subg_cols[wcut5] <- '#A0A0A0' # '5gray'
cell_subg_names[wcut5] <- '5gray'
cell_subg_cols[wcut6] <- '#C0C0C0' # '6gray'
cell_subg_names[wcut6] <- '6gray'
cell_subg_cols[wcut7] <- '#F0F0F0' # '6gray'
cell_subg_names[wcut7] <- '7gray'
names(cell_subg_cols) <- cim_o

## Plot the same fully clustered plot, this time without reclustering the cells,
# but coloring the horizontal color bar according to the cuts
pdf(f4, width = 8, height = 30)
cim_cells <- cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth,
    clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title4,
    col.sideColors = cell_subg_cols[wcuts],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_save$row.names]
colData(vsmcmod_cds)$subgroups[wcuts] <- cell_subg_names[wcuts]
pdf(f5, width = 3.75, height = 38)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "subgroups",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

## Look at the mean expression of the first three groups
# par(mfrow=c(1,3))
# hist(apply(log10(exprs(down_stdycds[, colnames(expdatbo)[wcut1]]) +.1), 1, mean))
# hist(apply(log10(exprs(down_stdycds[, colnames(expdatbo)[wcut2]]) +.1), 1, mean))
# hist(apply(log10(exprs(down_stdycds[, colnames(expdatbo)[wcut3]]) +.1), 1, mean))


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

## VSMC Unknown Process (Modules 9, 11, 13, 24, 28, 30, 31, plus CCL4 and IGKC)¶
### Updated code

In [222]:
## Get the Expression Set for VSMC Module 9, 11, 13, 24, 28, 30, 31, DE Genes, plus CCL4 and IGKC
p <- 3
vsmc_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
vsmcmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% vsmc_modgenes_9_11_13_24_28_30_31_plus2_ids, vsmc_bool]
colData(vsmcmod_cds)$condtype <- paste('vsmc',
    c('hea', 'dis')[(colData(vsmcmod_cds)$condition == 'diseased') + 1], sep = '-')
## colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-dis'] <- 'red'
## colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'ec-hea'] <- 'blue'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
colData(vsmcmod_cds)$sidecolor[colData(vsmcmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'

## Down Sample
cttable <- table(colData(vsmcmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(vsmcmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
vsmcmod_cds <- vsmcmod_cds[, cells2keep] 

## Print the down sampled data set sizes
table(colData(vsmcmod_cds)$sidecolor)

## Get the expresion matrix
expdat_vsmcmod <- as.matrix(exprs(vsmcmod_cds))
rownames(expdat_vsmcmod) <- geneid2short[rownames(expdat_vsmcmod)] 

## Set up the output file names and plot titles
f1 <- '/Heatmap.VSMC.Mod9.11.13.24.28.30.31.Plus2.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev3.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'VSMC Network Modules 9, 11, 13, 24, 28, 30, 31, Plus 2\nGenes, Cells Pre-Clustered'

f2 <- '/Heatmap.VSMC.Mod9.11.13.24.28.30.31.Plus2.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev3.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'VSMC Network Modules 9, 11, 24, 28, 13, 30, 31, Plus 2\nGenes, Cells Fully Clustered'

f3 <- '/DotPlot.VSMC.Mod9.11.13.24.28.30.31.Plus2.Genes.Rev3.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

f4 <- '/Heatmap.VSMC.Mod9.11.13.24.28.30.31.Plus2.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.cut.Rev5.pdf'
f4 <- paste(datdir, '/', ps$outdir, celltypes6[p], f4, sep = '')
title4 <-  'VSMC Network Modules 9, 11, 24, 28, 13, 30, 31, Plus 2\nGenes, Cells Fully Clustered with Cuts'

f5 <- '/DotPlot.VSMC.Mod9.11.13.24.28.30.31.Plus2.Genes.cuts.Rev5.pdf'
f5 <- paste(datdir, '/', ps$outdir, celltypes6[p], f5, sep = '')

## Prepare for pre-clustering
vsmc_dw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-dis')
vsmc_hw <- which(colData(vsmcmod_cds)$condtype == 'vsmc-hea')

## Transform the data
dims <- dim(expdat_vsmcmod)
expdatbo <- (expdat_vsmcmod > 1) * 1

## Set up clustering parameters
## Distance method, cluster method, heatmap color scheme, legend
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('VSMC DIS', 'VSMC HEA'),
              col = c('orange', 'cyan'), title = 'Cell Types', cex = .7)

## Individually cluster the columns of the two cell sets
vsmc_dcim <- cim(expdatbo[, vsmc_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_dw],
    symkey = FALSE)
vsmc_hcim <- cim(expdatbo[, vsmc_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colData(vsmcmod_cds)$sidecolor[vsmc_hw],
    symkey = FALSE)

## The is the combined (concantenated) preclustered cell types
## which will are plotted with genes fully clustered (check plot)
parthclust <- c(colnames(vsmc_dcim$mat), colnames(vsmc_hcim$mat))
colornames <-colData(vsmcmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)

## This is just a check plot to the screen
# cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[parthclust],
#     symkey = FALSE)

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 30)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_save <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## This is a check plot to the screen and it should match the above plot perfectly
# cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[colnames(cim_save$mat)], 
#     symkey = FALSE)

## Repeated from above, this time to a file (make sure to cluster again to plot cell dendrogram)
pdf(f2, width = 8, height = 30)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- short2geneid[cim_save$row.names]
pdf(f3, width = 2.75, height = 34)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

## Set up for plot with special cut regions
cim_o <- colnames(cim_save$mat)
cut0 <- 1
cut1 <- floor(length(cim_o) * (15 / 16) / 7.1)
cut2 <- floor(length(cim_o) * (1 + 11 / 16) / 7.1)
cut3 <- floor(length(cim_o) * (2 + 1 / 4) / 7.1)
cut4 <- floor(length(cim_o) * (3 + 5 / 8) / 7.1)
cut5 <- floor(length(cim_o) * (4 + 15 / 16) / 7.1)
cut6 <- length(cim_o)
wcut1 <- which(colnames(expdatbo) %in% cim_o[cut0:cut1])
wcut2 <- which(colnames(expdatbo) %in% cim_o[(cut1 + 1):cut2])
wcut3 <- which(colnames(expdatbo) %in% cim_o[(cut2 + 1):cut3])
wcut4 <- which(colnames(expdatbo) %in% cim_o[(cut3 + 1):cut4])
wcut5 <- which(colnames(expdatbo) %in% cim_o[(cut4 + 1):cut5])
wcut6 <- which(colnames(expdatbo) %in% cim_o[(cut5 + 1):cut6])
wcuts <- c(wcut1, wcut2, wcut3, wcut4, wcut5, wcut6)
cell_subg_cols <- vector(mode = 'character', length = length(cim_o))
cell_subg_names <- vector(mode = 'character', length = length(cim_o))
cell_subg_cols[wcut1] <- '#404040' # '1gray'
cell_subg_names[wcut1] <- '1gray'
cell_subg_cols[wcut2] <- '#606060' # '2gray'
cell_subg_names[wcut2] <- '2gray'
cell_subg_cols[wcut3] <- '#808080' # '3gray'
cell_subg_names[wcut3] <- '3gray'
cell_subg_cols[wcut4] <- '#A0A0A0' # '4gray'
cell_subg_names[wcut4] <- '4gray'
cell_subg_cols[wcut5] <- '#C0C0C0' # '5gray'
cell_subg_names[wcut5] <- '5gray'
cell_subg_cols[wcut6] <- '#E0E0E0' # '6gray'
cell_subg_names[wcut6] <- '6gray'
names(cell_subg_cols) <- cim_o

## Plot the same fully clustered plot, this time without reclustering the cells,
# but coloring the horizontal color bar according to the cuts
pdf(f4, width = 8, height = 30)
cim_cells <- cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth,
    clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title4,
    col.sideColors = cell_subg_cols[wcuts],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_save$row.names]
colData(vsmcmod_cds)$subgroups[wcuts] <- cell_subg_names[wcuts]
pdf(f5, width = 3.25, height = 34)
plot_genes_by_group2(vsmcmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "subgroups",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


  cyan orange 
  1223   1223 

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2

## EC Unknown Process (Modules 1, 2, 6,  35, 36)
### Updated code

In [223]:
## Get the Expression Set for EC 1, 2, 6, 35, 36 DE Genes
p <- 2
ec_bool <- (colData(down_stdycds)$assigned_cell_type == celltypes6[p])
ecmod_cds <- down_stdycds[fData(down_stdycds)@rownames %in% ec_modgenes_1_2_6_35_36_ids, ec_bool]
colData(ecmod_cds)$condtype <- paste('ec',
    c('hea', 'dis')[(colData(ecmod_cds)$condition == 'diseased') + 1], sep = '-')
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-dis'] <- 'red'
colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'ec-hea'] <- 'blue'
# colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-dis'] <- 'orange'
# colData(ecmod_cds)$sidecolor[colData(ecmod_cds)$condtype == 'vsmc-hea'] <- 'cyan'

## Down Sample
cttable <- table(colData(ecmod_cds)$sidecolor)
smallset <- min(cttable)
cells2keep <- NULL
set.seed(101)
for (cellset_col in names(cttable)) {
    cellsetrow <-  which(colData(ecmod_cds)$sidecolor == cellset_col)
    cells2keep <- c(cells2keep, sample(cellsetrow, smallset, replace = FALSE))
}
ecmod_cds <- ecmod_cds[, cells2keep] 

## Print the down sampled data set sizes
table(colData(ecmod_cds)$sidecolor)

## Get the expresion matrix
expdat_ecmod <- as.matrix(exprs(ecmod_cds))
rownames(expdat_ecmod) <- geneid2short[rownames(expdat_ecmod)] 

## This is a new round of heatmaps, with new genes and down sampled cell tpyes
f1 <- '/Heatmap.EC.Mod1.2.6.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.preclustered.Rev2.pdf'
f1 <- paste(datdir, '/', ps$outdir, celltypes6[p], f1, sep = '')
title1 <- 'Endothelial Network \nModules 1, 2, 6, 35 and 36 Genes, Cells Pre-Clustered'

f2 <- '/Heatmap.EC.Mod1.2.6.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.Rev2.pdf'
f2 <- paste(datdir, '/', ps$outdir, celltypes6[p], f2, sep = '')
title2 <-  'Endothelial Network \nModule  1, 2, 6, 35 and 36 Genes, Cells Fully Clustered'

f3 <- '/DotPlot.EC.Mod1.2.6.35.36.Genes.Rev2.pdf'
f3 <- paste(datdir, '/', ps$outdir, celltypes6[p], f3, sep = '')

f4 <- '/Heatmap.EC.Mod1.2.6.35.36.Genes.comp.bin.comp.bin.down.OnOff2Plus.fullyclustered.cut.Rev5.pdf'
f4 <- paste(datdir, '/', ps$outdir, celltypes6[p], f4, sep = '')
title4 <-  'VSMC Network Modules  1, 2, 6, 35 and 36\nGenes, Cells Fully Clustered with Cuts'

f5 <- '/DotPlot.EC.Mod1.2.6.35.36.Genes.cuts.Rev5.pdf'
f5 <- paste(datdir, '/', ps$outdir, celltypes6[p], f5, sep = '')

## Prepare for pre-clustering
ec_dw <- which(colData(ecmod_cds)$condtype == 'ec-dis')
ec_hw <- which(colData(ecmod_cds)$condtype == 'ec-hea')

## Transform the data
dims <- dim(expdat_ecmod)
expdatbo <- (expdat_ecmod > 1) * 1

## Set up clustering parameters
## Distance method, cluster method, heatmap color scheme, legend
distmeth <- c('binary', 'binary')
clustmeth = c('complete', 'complete')
my_palettebol <- colorRampPalette(c('#660066', 'yellow'))(n = 2)
legendlstbol <- list(legend = c('EC DIS', 'EC HEA'),
              col = c('red', 'blue'), title = 'Cell Types', cex = .7)

## Individually cluster the columns of the two cell sets
ec_dcim <- cim(expdatbo[, ec_dw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_dw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')
ec_hcim <- cim(expdatbo[, ec_hw], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'col',
    col.sideColors = colData(ecmod_cds)$sidecolor[ec_hw],
    symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## This is the combined (concantenated) preclustered cell types
## which are plotted with genes fully clustered (check plot)
parthclust <- c(colnames(ec_dcim$mat), colnames(ec_hcim$mat))
colornames <-colData(ecmod_cds)$sidecolor
names(colornames) <- colnames(expdatbo)

## This is just a check plot to the screen
# cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row',
#     col.sideColors = colornames[parthclust],
#     symkey = FALSE, save = 'pdf', name.save = 'temp.ready.to.delete')

## Repeated from above, this time to a file
pdf(f1, width = 8, height = 24)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title1,
    col.sideColors = colornames[parthclust],
    symkey = FALSE)
dev.off()

## The is again the combined cell types, plotted with both genes and cells fully clustered
cim_save <- cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', save = 'pdf', name.save = 'temp.ready.to.delete',
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)

## This is a check plot to the screen and it should match the above plot perfectly
# cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth, clust.method = clustmeth,
#     scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
#     color = my_palettebol, cluster = 'row', # save = 'pdf', name.save = 'temp.ready.to.delete',
#     col.sideColors = colornames[colnames(cim_save$mat)], 
#     symkey = FALSE)

## Repeated from above, this time to a file (make sure to cluster again to plot cell dendrogram)
pdf(f2, width = 8, height = 24)
cim(expdatbo[, parthclust], dist.method = distmeth, clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'both', title = title2,
    col.sideColors = colornames[parthclust], 
    symkey = FALSE)
dev.off()

specialglist <- short2geneid[cim_save$row.names]
pdf(f3, width = 2.75, height = 23)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "condtype",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()

## Set up for plot with special cut regions
cim_o <- colnames(cim_save$mat)
cut0 <- 1
cut1 <- floor(length(cim_o) * (1.573) / 5.664)
cut2 <- floor(length(cim_o) * (3.205) / 5.664)
cut3 <- length(cim_o)
wcut1 <- which(colnames(expdatbo) %in% cim_o[cut0:cut1])
wcut2 <- which(colnames(expdatbo) %in% cim_o[(cut1 + 1):cut2])
wcut3 <- which(colnames(expdatbo) %in% cim_o[(cut2 + 1):cut3])
wcuts <- c(wcut1, wcut2, wcut3)
cell_subg_cols <- vector(mode = 'character', length = length(cim_o))
cell_subg_names <- vector(mode = 'character', length = length(cim_o))
cell_subg_cols[wcut1] <- '#404040' # '1gray'
cell_subg_names[wcut1] <- '1gray'
cell_subg_cols[wcut2] <- '#606060' # '2gray'
cell_subg_names[wcut2] <- '2gray'
cell_subg_cols[wcut3] <- '#808080' # '3gray'
cell_subg_names[wcut3] <- '3gray'
names(cell_subg_cols) <- cim_o

## Plot the same fully clustered plot, this time without reclustering the cells,
# but coloring the horizontal color bar according to the cuts
pdf(f4, width = 8, height = 24)
cim_cells <- cim(expdatbo[, colnames(cim_save$mat)], dist.method = distmeth,
    clust.method = clustmeth,
    scale = "FALSE", center = "FALSE", col.names = FALSE, legend = legendlstbol,
    color = my_palettebol, cluster = 'row', title = title4,
    col.sideColors = cell_subg_cols[wcuts],
    symkey = FALSE)
dev.off()

specialglist <- specialglist <- short2geneid[cim_save$row.names]
colData(ecmod_cds)$subgroups[wcuts] <- cell_subg_names[wcuts]
pdf(f5, width = 3.25, height = 27)
plot_genes_by_group2(ecmod_cds,
                    axis_order = "group_marker",
                    specialglist,
                    group_cells_by = "subgroups",
                    ordering_type = "as_ordered", 
                    max.size = 3,
                    norm_method = 'size_only',
                    scale_max = 3, scale_min = -3)
dev.off()


blue  red 
 449  449 

png 
  2

png 
  2

png 
  2

png 
  2

png 
  2